In [ ]:
import os 

import copy
from pathlib import Path
import warnings

import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import math

In [ ]:
from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer, EncoderNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss, MAPE, RMSE, MAE, NormalDistributionLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

In [ ]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import torch.nn.functional as F

In [ ]:
import torch.nn as nn
import torch.optim as optim

import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile

In [ ]:
# Directorio donde se encuentra la base de datos con los índices x serie
os.chdir('C:\\Users\\ALVARLX23\\OneDrive - Abbott\\Documents\\Proyecto\\Redes Neuronales')
os.getcwd()

In [ ]:
# Leer datos y convertir variables en categorias
DATA= pd.read_excel('data.xlsx') 
DATA['timeseries']=pd.Categorical(DATA.timeseries) 
DATA['brand']=pd.Categorical(DATA.brand)
DATA['channel']=pd.Categorical(DATA.channel)
DATA['units'] = DATA['units'].astype(float)

In [ ]:
# Añadir índice de tiempo
DATA["time_idx"] = DATA["YearMonth"].dt.year * 12 + DATA["YearMonth"].dt.month
DATA["time_idx"] -= DATA["time_idx"].min()

In [ ]:
# Ver el tamaño y la clase de las variables y el data frame
print(DATA.shape)
print(DATA.dtypes)
type(DATA)

# Ver las primeras observaciones
DATA.head()

In [ ]:
# Adicionar variables 
DATA["month"] = DATA.YearMonth.dt.month.astype(str).astype("category")  # Las categorias tienen que ser strings
DATA["avg_volume_by_sku"] = DATA.groupby(["time_idx", "sku"], observed=True).units.transform("mean") # la misma fecha x sku (promedio entre los dos canales)
DATA["avg_volume_by_channel"] = DATA.groupby(["time_idx", "channel"], observed=True).units.transform("mean") # todos los de la misma fecha x canal
DATA["avg_volume_by_brand"] = DATA.groupby(["time_idx", "brand"], observed=True).units.transform("mean") # todos los de la misma fecha x canal


In [ ]:
# Meses de prueba + 28 Fcst
max_prediction_length = round(DATA.shape[0]/np.unique(DATA.timeseries).shape[0]*0.1) + (28 - round(DATA.shape[0]/np.unique(DATA.timeseries).shape[0]*0.1))
# Meses de entrenamiento (90% datos = que con ARIMA)
max_encoder_length = round(DATA.shape[0]/np.unique(DATA.timeseries).shape[0]*0.9)
training_cutoff = DATA["time_idx"].max() - (max_prediction_length - (28 - round(DATA.shape[0]/np.unique(DATA.timeseries).shape[0]*0.1)))

training = TimeSeriesDataSet(
    DATA[lambda x: x.time_idx <= training_cutoff],
    time_idx="time_idx",
    target="units",
    group_ids=["timeseries", "channel"],
    min_encoder_length=max_encoder_length // 2,  
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    static_categoricals=["channel", "sku", "brand"],
    static_reals=["SAP"],
    time_varying_known_categoricals=["month"],
    time_varying_known_reals=["time_idx"],
    time_varying_unknown_categoricals=[],
    time_varying_unknown_reals=[
        "units",
        "avg_volume_by_channel",
        "avg_volume_by_sku",
        "avg_volume_by_brand",
    ],
    target_normalizer=GroupNormalizer(
        groups=["timeseries","channel"], transformation="softplus" #
    ),  # Usa softplus para normalizar por grupos
    add_relative_time_idx=True,
    add_target_scales=True,
    allow_missing_timesteps=True
    #add_encoder_length=True,
)

# crear conjunto de validación (predict=True) que significa predecir los max_prediction_length puntos en el tiempo
# para cada serie
validation = TimeSeriesDataSet.from_dataset(training, DATA, predict=True, stop_randomization=True)

# crear dataloaders para el modelo
batch_size = 32  # En general 32 y probar bajando y subiendo por potencias de 2
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=4)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=4)

In [ ]:
# calcular MAE base. las predicciónes quedan como el último valor disponible (esto para tener un punto de referencia)
actuals = torch.cat([y for x, (y, weight) in iter(val_dataloader)])
baseline_predictions = Baseline().predict(val_dataloader)
(actuals[:, 0:7] - baseline_predictions[:,0:7]).abs().mean().item()

In [ ]:
# configurar la red y el entrenador
pl.seed_everything(42)
trainer = pl.Trainer(
    gpus=0,
    # fijar el gradiente es un hiperparámetro y es importante para evitar la divergencia
    # del gradiente para una red neuronal recurrente
    gradient_clip_val=0.2,
)


tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.03,
    hidden_size=8,  
    attention_head_size=1,
    dropout=0.2,  # entre 0.1 y 0.3 son buenos valores
    hidden_continuous_size=8,  # set to <= hidden_size
    output_size=1,  
    loss=QuantileLoss([0.5]), # utilizar la mediana como predictor
    # reducir la tasa de aprendizaje si no hay mejora en la pérdida de validación después de x epochs
    reduce_on_plateau_patience=2,
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

In [ ]:
# Encontrar la tasa de aprendizaje óptima (suele dar una demasiado pequeña)
res = trainer.tuner.lr_find(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
    max_lr=10.0,
    min_lr=1e-6,
)

print(f"suggested learning rate: {res.suggestion()}")
fig = res.plot(show=True, suggest=True)
fig.show()

In [ ]:
# Configurar la red y el entrenador teniendo en cuenta la tasa de aprendizaje
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
lr_logger = LearningRateMonitor()  # log de la tasa de aprendizaje
logger = TensorBoardLogger("lightning_logs")  # logging results to a tensorboard

trainer = pl.Trainer(
    max_epochs=30,
    gpus=0,
    weights_summary="top",
    gradient_clip_val=0.2,
    limit_train_batches=32, 
    callbacks=[lr_logger, early_stop_callback],
    logger=logger,
    log_every_n_steps=16,
)


tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.15,
    hidden_size=16,
    attention_head_size=1,
    dropout=0.2,
    hidden_continuous_size=8,
    output_size=1,  
    loss=QuantileLoss([0.5]),
    log_interval=10,  
    reduce_on_plateau_patience=4,
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

In [ ]:
# Ajustar red (este se demora en correr dado que es donde se hace el ajuste del modelo)
trainer.fit(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

In [ ]:
# Cargar el mejor modelo según la pérdida de validación
# (como se usó early stopping no necesariamente es el último epoch)
best_model_path = trainer.checkpoint_callback.best_model_path
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)

In [ ]:
# EXPLICA QUE TIPI DE PARAMETROS Y CUANTOS PARAMETROS AJUSTO EL MODELO
best_tft.summarize()

In [ ]:
# calcular error medio absoluto 
actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
predictions = best_tft.predict(val_dataloader)
(actuals[:,0:7] - predictions[:,0:7]).abs().mean()

In [ ]:
x, y = next(iter(training.to_dataloader(batch_size=32)))
y[0]  # Lista de valores target

In [ ]:
out = best_tft(x)
out

best_tft.loss(out["prediction"], y)

In [ ]:
# raw predictions: diccionario con todo tipo de info (ej. cuantiles)
raw_predictions, x = best_tft.predict(val_dataloader, mode="raw", return_x=True)

In [ ]:
# Solo es información de que variables tuvo en cuenta
interpretation = best_tft.interpret_output(raw_predictions, reduction="sum")
best_tft.plot_interpretation(interpretation)

In [ ]:
# realizar las predicciones con el mejor modelo para el conjunto de validación
predictions = best_tft.predict(val_dataloader)
predictions.shape

In [ ]:
# convertirpredicciones de un tensor a un data frame
y_pred=pd.DataFrame(predictions[:,0:7].numpy()) 
y_pred.index = np.unique(DATA['timeseries'])
y_pred.head(5)

### Métricas

In [ ]:
pred_prueba = predictions[:,0:7]
actual_prueba = actuals[:, 0:7]

In [ ]:
# RMSE
a = ((actual_prueba - pred_prueba)**2).mean(axis=1)
rmse = []

for i in a:
    rmse.append(math.sqrt(i))
print(rmse)

# MAPE
mape= ((actual_prueba - pred_prueba).abs()/actual_prueba).mean(axis=1)

In [ ]:
np.unique(DATA['sku']).shape # revisar cuando vuelva a correrlo

In [ ]:
# Adecuación para que salga el Data Frame de métricas
metricas = pd.DataFrame({"time Serie": np.unique(DATA['timeseries']), "mape": mape.numpy(),"rmse": np.asarray(rmse)})
metricas.head(10)

### Predicción en nuevos datos

In [ ]:
encoder_data = DATA[lambda x: x.time_idx > x.time_idx.max() - max_encoder_length]

# Seleccionar los últimos puntos conocidos y creamos el decoder a partir de allí repitiendo incrementando la fecha
last_data = DATA[lambda x: x.time_idx == x.time_idx.max()]

decoder_data = pd.concat(
    [last_data.assign(YearMonth=lambda x: x.YearMonth + pd.offsets.MonthBegin(i)) for i in range(1, max_prediction_length + 1)],
    ignore_index=True,
)


In [ ]:
# ESTA SE VA A CORTAR, PERO POR AHORA NO HASTA ESTAR SEGURA AL CORRERLO AGAIN

print(min(encoder_data.YearMonth),
      max(encoder_data.YearMonth),
      min(decoder_data.YearMonth),
      max(decoder_data.YearMonth),
      len(encoder_data), len(decoder_data), sep = "\t"
     )


In [ ]:
# Añadir índice de tiempo consistente con la data
decoder_data["time_idx"] = decoder_data["YearMonth"].dt.year * 12 + decoder_data["YearMonth"].dt.month
decoder_data["time_idx"] += encoder_data["time_idx"].max() + 1 - decoder_data["time_idx"].min()

# ajustar características de tiempo
decoder_data["month"] = decoder_data.YearMonth.dt.month.astype(str).astype("category")  # categories have be strings

# unir enconder y decoder
new_prediction_data = pd.concat([encoder_data, decoder_data], ignore_index=True)

In [ ]:
new_raw_predictions, new_x = best_tft.predict(new_prediction_data, mode="raw", return_x=True)

for idx in range(10):  # plot 10 examples
    best_tft.plot_prediction(new_x, new_raw_predictions, idx=idx, show_future_observed=False);


In [ ]:
# 155 -> PEDIASURE CLINICAL RPB 220ML 
# 121 -> ENSURE BASE RPB VAINILLA
best_tft.plot_prediction(new_x, new_raw_predictions, idx=123, show_future_observed=False)

In [ ]:
# realizar las predicciones con el mejor modelo para el conjunto de validación
predictions_new = best_tft.predict(new_prediction_data)
predictions_new.shape

In [ ]:
# convertirpredicciones de un tensor a un data frame
y_new=pd.DataFrame(predictions_new.numpy()) 
y_new.index = np.unique(decoder_data['timeseries'])
y_new.head(5)

In [ ]:
# Exportar a excel(analogo a R)
writer = pd.ExcelWriter('C:\\Users\\ALVARLX23\\OneDrive - Abbott\\Documents\\Proyecto\\Redes Neuronales\\salida_redes.xlsx')
y_pred.to_excel(writer, sheet_name="estimación", index=True)
metricas.to_excel(writer, sheet_name="metricas", index=False)
y_new.to_excel(writer, sheet_name="28 meses", index=True)
writer.save()
writer.close()